# DCM Tutorial 

The dynamic causal model, first presented by Friston, et al. xxxx, provides a means of modeling the effective connectivity between neural populations that underlies task based or resting state functional neuroimaging data as measured through fMRI, EEG, and MEG. 

## Model Components

The model consists of three main parts:

-  **Dynamic Causal Model:** A model that combines a bilinear model of neuronal dynamics with a neuroimaging method - specific model (based on hemodynamic or electrophysiological signatures). Proposed brain regions and potential connections between regions are specified in one or more candidate models.
-  **Variational Laplace:** The process by which candidate model parameters are optimized and their uncertainty is determined. Approximates the log marginal likelihood for each model using variational free energy. Comparing the free energy of candidate models enalbes the model that best explains the data to be selected.
-  **Model Selection & Comparison:** The process by which model structure is selected. There are two common approaches:
    - **Bayesian Model Selection (BMS):** Multiple DCM models are optimized, where each describes a hypothesis about the connectivity between brain regions of interest. The model that best describes each participant's data is determined and the average connectivity parameters are determined using Bayesian Model Averaging, with each parameter a weighted average of the candidate models based on each's prior probability.
    - **Parametric Empirical Bayes (PEB):** Assumes that each participant varies in the strength of connection between regions. Models the soruces of variability in connection strengths across subjects using fixed and random effects. Specifies one DCM per subject and a Bayesian GLM to model parameters at the group level. Tests hypotheses by comparing the group level model to reduced group level models where certain combinations of connections are switched on and off to determine the best fit for the data. 

## General DCM Model: Mathematical Framework 

This model produces a deterministic model of neurla dynamics in a network of n interacting brain regions/nodes. 

The model transforms a neuronal state vector $\vec{x}$ where each region is represented by a single hidden state using the bilinear differential equation:

$\frac{dx}{dt} = f(x, u, \theta) = Ax + \Sigma_{j=1}^m u_j B^{(j)} x + Cu$ 

where

$u$ represents the external or exogenous input.

$A = \frac{\del f}{\del x} |_{u=0}$ represents the fixed (average) coupling among nodes in the absence of exogenous output $u(t)$. This roughly represents latent coupling among nodes in the absence of experimental perturbations.

$B = \frac{\del^2 f}{\del x \del u}$ represents the change in latent coupling induced by the jth input, which encodes context sensitive changes in A, or modulations of coupling by experimental perturbations. 

$C = \frac{\del f}{\del u} |_{x = 0}$ represents the influence of exogneous inputs that cause perturbations in hidden states. 

The model results from a bilinear Taylor approximation to any dynamical model of how changes in neuronal activity in node x_i are caused by changes in the others. Parameters $A, B, C \in \theta$ are coupling parameters we seek to find that determine the functional architecture and interactions between brain regions at a neuronal level. 

## DCM for fMRI: Hemodynamic Model 

This model describes the transformation of neural activity into a BOLD response. The common hemodynamic model uses the "balloon model", which uses the analogy of an inflated balloon to describe the behavior of the postcapillary venous component of the blood vessels "feeding" the neurons to encourage activity. The model postulates a nonlinear dependence of the BOLD signal on blood volume $V$ and deoxyhemoglobin content $q$. It explores the evolution of $V$ and $q$ by two differential equations operating under the assumptions that: 

- Small post capillary blood vessels releact to an influx of blood like an inflating balloon. So, changes in normalized blood volume $V$ correspond to differences in flow $f_i$ and outflow $f_o$ with some time constant $\tau$ where $\tau$ represents the mean transit time for blood to traverse the venous compartment such that $ \tau = \frac{V_0}{f_0} = \frac{\text{resting blood volume}}{\text{resting blood flow}}$. Outflow $f_o$ is modelled as a function of volume with parameter $\alpha$ the resistance of the venous balloon: $f_o = v^{1/\alpha}$.

- Oxygen extraction is highly coupled to blood flow, so it determines the deoxyhemoglobin content $q$. Generally a change in $q$ refers to a delivery of deoxyhemoglobin into the venous compartment minus that expelled at a given time point. So, the oxygen gradient across the capillary wall and oxygen extraction rate depend entirely on oxygen delivery, and as a result, blood flow.

These assumptions lead to state equations for $V$ and $q$:

$\tau \frac{dV}{dt} = f_i(t) - v(t)^{1/\alpha}$

$\tau \frac{dq}{dt} = f_i(t) ( \frac{1 - (1 - E_0)^{1/f_i}}{E_0}) - v(t)^{1/\alpha}(\frac{q(t)}{(v(t)}$

where $E_0$ is oxygen extraction fraction at rest. 

The hemodynamic model also has vascular state equations that address how bloodflow depends on neural activity, connecting to the neural state vector $x$. These vascular responses to neural activity correspond to a damped oscillator: changes in neural activity $x$ elicit exponentially decaying vasodilatory signal $s$ subject to feedback regulation by the flow $f$ which is normalized to resting flow $F_0$ that it induces. This results in two equations:

$ \frac{ds}{dt} = x - \kappa s - \gamma(f-1)$

$\frac{df}{dt} = s$ 

where $x$ is the neural state vector, $\kappa$ the rate constant of signal decay, and $\gamma$ the rate constant of feedback regulation. 

Several variants of the BOLD model itself, which unites parameters determined by the prior equations, but all are variations on a general model proposed by Stephen et al. 2007: 

$ \frac{\delta s}{s_0} = v_0 ( k_1(1 - q) + k_2(1 - \frac{q}{v}) + k_3(1 - v))$ 

where 

$k_1 = 4.3 \upsilon_0 E_0 T E$

$k_2 = \epsilon r_0 E_0 TE$

$k_3 = 1 - \epsilon$

and $\upsilon_0$ is the frequency offset for fully deoxygenated blood ($\upsilon = 40.3 s^{-1}$ at 1.5 teslas, $\epsilon$ is the ratio of intra and exogenous signal, TE is the time elapsed between when protons are excited and measured (how long signal creation takes), and $s_0$ is the BOLD signal at rest, described by:

$s_0 = (1 - V_0)S_E + V_0S_I$

where $V_0$ is the resting venous volume fraction, $S_E$ is the extravenous signal, and $S_I$ the intravascular signal. 

For any combination of parameters $\theta$ and inputs $u$, the measured BOLD response $y$ is modelled as a predicted BOLD signal $h(x, u, \theta)$ plus a linear combination of confounds $X\beta$ and Gaussian observation error:

$y = h(x, u, \theta) + X\beta + \epsilon$

## DCM for EEG: Electrophysiological Model

Originally presented in David, Kiebel, et al. 2006.